In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense,StringLookup,GlobalAveragePooling1D
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

import os
import pandas as pd
physical_devices = tf.config.list_physical_devices('GPU') 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.config.experimental.set_memory_growth(physical_devices[0], True) 

file_path = os.path.join(os.getcwd(),'data_sets', 'all_Article_df.parquet')
print(file_path)
df = pd.read_parquet(file_path)
print(df.columns)
# Split the DataFrame based on the label


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

df_dict = dict(tuple(df.groupby('label')))
maxArticle=0
label_word_frequency = {}
lemmatizer = WordNetLemmatizer()


def preprocess_text(text):
  # Lowercase the text
  text = tf.strings.lower(text)
  # Remove punctuation
  text = tf.strings.regex_replace(text, r"[^a-zA-Z0-9\s]", "")
  # You can add more preprocessing steps like stemming or lemmatization (optional)
  #text= tf.strings.split(text)
  #text = [lemmatizer.lemmatize(word.numpy().decode('utf-8')) for word in text]
  #text = tf.strings.join(text)


  return text.numpy().decode('utf-8')

max_article_length=0
tokenizer = Tokenizer()
count=0
breakflag=False
for label, articles_df in df_dict.items():
    word_counts = tf.zeros(len(tokenizer.word_index), dtype=tf.int32)
    for article in articles_df['Article']:
        max_article_length=max(max_article_length,len(article))
        count+=1
        print(count, end='\r', flush=True)
        processed_article = preprocess_text(article)
        if count>=854:
            break
        article_tokens = tokenizer.texts_to_sequences([processed_article])[0]
        # Update word counts using the tokens
        word_counts += tf.math.bincount(article_tokens)
    # Split the article into words
    count=0
    label_word_frequency[label] = word_counts
    if breakflag:
        break
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Article'])
sequences = tokenizer.texts_to_sequences(df['Article'])


In [ ]:

vocabulary_size = len(set(word for word_counts in label_word_frequency.values() for word in word_counts))
embedding_dim = 128  # You can adjust this hyperparameter
num_labels = len(df_dict)

# Create the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dim))  # Adjust max_article_length
model.add(Flatten())
model.add(Dense(num_labels, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 7: Compile the model


In [ ]:
def pad_and_batch(sequences):
  padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_article_length, padding='post')
  return padded_sequences
labels = df['label'].to_numpy()

batch_size = 32  # Adjust batch size
dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
dataset = dataset.batch(batch_size)
for epoch in range(10):
  for batch_sequences, batch_labels in dataset:
    padded_sequences = pad_and_batch(batch_sequences)
    model.fit(padded_sequences, batch_labels, epochs=1) 